# Notebook para geração de VHDL para SAD

In [157]:
import numpy as np 
import math

In [158]:
################################################
################## Parametros ##################
block_size = 16
bit_width = 8
proc_parallel = True                            # True: processamento paralelo de bloco todo, False: processamento por linha
files = ['carry_lookahead_adder.vhd']           # Lista de componentes para add
# files = ['adder.vhd']
adder_name = 'carry_lookahead_adder'
# adder_name = 'adder'                          # Nome do somador (só pra facilitar)
write_vhd = True

if proc_parallel:
    file_name = ('cl_sad_' + str(int(math.sqrt(block_size))) + 'x' + str(int(math.sqrt(block_size))) + '_parallel')
else:
    file_name = ('cl_sad_' + str(int(math.sqrt(block_size))) + 'x' + str(int(math.sqrt(block_size))) + '_line')

pack_name = 'type_pack'
rewrite_package = False
################################################

In [159]:
################################################# Basis #################################################
if proc_parallel:                           # Ex: bloco 4x4 processa 16 em paralelo
    limit = block_size
else:
    limit = int(math.sqrt(block_size))      # Ex: bloco 4x4 processa 4 vezes cada linha de 4

################################################# Port ################################################
port = ('''ori: in input_'''+str(limit)+''';
    ref: in input_'''+str(limit)+''';
    sad_out: out std_logic_vector(''' + str(math.ceil(np.log2(block_size))+bit_width) + ''' downto 0)''')

################################################# Components ################################################
components = []
line_qnt = 0

for comp in files:
    with open(comp, "r") as file:
        lines = file.readlines()

    line_split = []
    for i in range(len(lines)):                # Separa os componentes de cada linha lida
        line_split.append(lines[i].split())

    end_line = 0
    for i in range(0,len(lines),1):                 # Itera em todas linhas        
        if line_split[i] != []:                     # Verifica se não é uma linha vazia
            if line_split[i][0] == 'entity':        # Vê se é entity                
                begin_line = i
            elif line_split[i][0] == 'end':         # Vê se é o final de entity
                if end_line == 0:
                    end_line = i

    for l in range(begin_line,end_line+1,1):        # Pega todas as linhas da entity
        components.append(lines[l])

    line1_split = components[line_qnt].split()      # Alterando a palavra 'entity' 
    line1_split[0] = 'component'                    # para 'component'
    line1_split.append('\n')
    components[line_qnt] = ' '.join(line1_split)

    linelast_split = components[-1].split()         # Alterando a última palavra 
    linelast_split[-1] = 'component;'               # para 'component'
    components[-1] = ' '.join(linelast_split)

    components.append('\n\n')
    line_qnt = len(components)

components_concat = ' '.join(components)         #  Junta todos componentes

################################################# Signals ################################################
negs = []   # Sinais negativos reference para entrar em somador e virar subtracao
for i in range(0,limit):
    negs.append('signal neg_ref_'+str(i)+': std_logic_vector('+str(bit_width-1)+' downto 0);')
negs.append('\n')
negs_concat = '\n'.join(negs)

sub = []    # Sinais da 1a subtracao (estrutura basica)
for i in range(0,limit):
    sub.append('signal sub_' + str(i) + ': std_logic_vector(' + str(bit_width) + ' downto 0);')
sub.append('\n')
sub_concat = '\n'.join(sub)

abs = []        # Sinais do calculo do absoluto (estrutura basica)
out_abs = []    # Lista com nome dos sinais que saem do absoluto
for i in range(0,limit):
    out_abs.append('abs_'+str(i))
    abs.append('signal abs_' + str(i) + ': std_logic_vector(' + str(bit_width) + ' downto 0);')
abs.append('\n')
abs_concat = '\n'.join(abs)

sum0 = []       # Sinais da 1a soma (estrutura basica)
out_sum = []    # Lista com nome dos sinais que saem da 1a soma
for i in range(0,limit,2):
    out_sum.append('sum_'+str(i) + str(i+1))
    sum0.append('signal sum_' + str(i) + str(i+1) + ': std_logic_vector(' + str(bit_width+1) + ' downto 0);')
sum0.append('\n')
sum0_concat = '\n'.join(sum0)

after_sums = []         # Sinais dos outros niveis de soma abaixo 
out_after_sums = []     # Lista com nome dos sinais que saem das outras somas
for k in range(1,math.ceil(np.log2(limit))):    # Qtd de niveis de soma
    qtd_sum = int(limit/(2**(k+1)))             # Qtd de somadores por nivel
    for j in range(0,qtd_sum):
        after_sums.append('signal sum_after_'+str(k)+str(j)+': std_logic_vector('+str(bit_width+k+1)+' downto 0);')
        out_after_sums.append('sum_after_'+str(k)+str(j))
after_sums.append('\n')
after_sums_concat = '\n'.join(after_sums)

signals = []
signals.append(negs_concat + sub_concat + abs_concat + sum0_concat + after_sums_concat)
signals_concat = ''.join(signals)
# print(signals_concat)

################################################# Assign ################################################
neg_assign = []     # Definindo os fios negativos pra entrar no somador (= subtrator)
for i in range(0,limit):
    neg_assign.append('neg_ref_'+str(i)+' <= std_logic_vector(resize(-signed(ref('+str(i)+')),'+str(bit_width)+'));')
neg_assign.append('\n')
neg_assign_concat = '\n'.join(neg_assign)

abs_assign = []     # Definindo os fios que saem do absoluto
for i in range(0,limit):
    abs_assign.append('abs_'+str(i)+' <= std_logic_vector(abs(signed(sub_'+str(i)+')));')
abs_assign.append('\n')
abs_assign_concat = '\n'.join(abs_assign)

assigns = []        # Juntando todos os assigns
assigns.append(neg_assign_concat + abs_assign_concat)
assigns_concat = ''.join(assigns)

# print(out_sum)
# print(out_after_sums)

In [160]:
################################################# Port maps #################################################
sub_map = []            #  Port map dos componentes subtratores
for i in range(0,limit):
    sub_map.append('SUB'+str(i)+': '+adder_name+' generic map ('+str(bit_width)+') port map (ori('+str(i)+'),neg_ref_'+str(i)+',sub_'+str(i)+');')
sub_map.append('\n')
sub_map_concat = '\n'.join(sub_map)
# print(sub_map_concat)

sums_map = []
count = 0
for i in range(0,limit,2):
    sums_map.append('ADD0'+str(count)+': '+adder_name+' generic map ('+str(bit_width+1)+') port map ('+out_abs[i]+','+out_abs[i+1]+','+out_sum[count]+');')
    count = count + 1
sums_map.append('\n')
sums_map_concat = '\n'.join(sums_map)
# print(sums_map_concat)

after_sums_map = []     # Sinais dos outros niveis de soma abaixo 
n = 0 
m = 0
l = 0
count = 0
for k in range(1,math.ceil(np.log2(limit))+1):          # Qtd de niveis de soma         2
    qtd_sum_map = int(limit/(2**(k+1)))                 # Qtd de somadores por nivel    2, 1
    for j in range(0,qtd_sum_map):
        if k == 1:
            after_sums_map.append('ADD'+str(k)+str(j)+': '+adder_name+' generic map ('+str(bit_width+k+1)+') port map ('+out_sum[count]+','+out_sum[count+1]+','+out_after_sums[n]+');')
            count = count + 2
            n = n + 1
        elif k >= 2: # 8 somadores, 4, 2, 1
            after_sums_map.append('ADD'+str(k)+str(j)+': '+adder_name+' generic map ('+str(bit_width+k+1)+') port map ('+out_after_sums[m]+','+out_after_sums[m+1]+','+out_after_sums[l+qtd_sum_map*(2**(k-1))]+');')
            m = m + 2
            l = l + 1
            
after_sums_map.append('\n')
after_sums_map_concat = '\n'.join(after_sums_map)

port_map = []
port_map.append(sub_map_concat + sums_map_concat + after_sums_map_concat)
port_map_concat = ''.join(port_map)

################################################# Final #################################################
if out_after_sums:
    result = out_after_sums[-1]
else:
    result = out_sum[-1]


In [161]:
############################################ Texto do VHDL ############################################
vhdl = ('''library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_unsigned.all;
use ieee.numeric_std.all;
use work.''' + pack_name + '''.all;

entity ''' + file_name + ''' is 
port(
    ''' + port + '''
);
end ''' + file_name + ''';

architecture arch_sad of ''' + file_name + ''' is 

-- Components 
''' + components_concat + '''
-- Signals
''' + signals_concat + '''
begin

''' + assigns_concat + '''
''' + port_map_concat + '''
sad_out <= ''' + result + ''';

end arch_sad;'''
)

############################################ File .vhd ############################################
if write_vhd:
    vhd_file = open((file_name + ".vhd"), "w")
    vhd_file.write(vhdl)
    vhd_file.close()


## Geração de Testbench

In [162]:
#################################### Parametros para testbench ####################################
tb_name = ('tb_'+file_name)

#################################### Component para tb ####################################
top_component = []
line_qnt = 0

with open((file_name+'.vhd'), "r") as top_file:
    top_lines = top_file.readlines()

line_split = []
for i in range(len(top_lines)):                # Separa os componentes de cada linha lida
    line_split.append(top_lines[i].split())

end_line = 0
for i in range(0,len(top_lines),1):                 # Itera em todas linhas        
    if line_split[i] != []:                     # Verifica se não é uma linha vazia
        if line_split[i][0] == 'entity':        # Vê se é entity                
            begin_line = i
        elif line_split[i][0] == 'end':         # Vê se é o final de entity
            if end_line == 0:
                end_line = i

for l in range(begin_line,end_line+1,1):        # Pega todas as linhas da entity
    top_component.append(top_lines[l])

line1_split = top_component[line_qnt].split()      # Alterando a palavra 'entity' 
line1_split[0] = 'component'                    # para 'component'
line1_split.append('\n')
top_component[line_qnt] = ' '.join(line1_split)

linelast_split = top_component[-1].split()         # Alterando a última palavra 
linelast_split[-1] = 'component;'               # para 'component'
top_component[-1] = ' '.join(linelast_split)

top_component.append('\n\n')
line_qnt = len(top_component)

top_component_concat = ' '.join(top_component)         #  Junta todos componentes

#################################### Signals ####################################
# begin_sig = begin_line + 2
# end_sig = end_line - 2
tb_signal = ('signal s_ori: input_'+str(limit)+';\n'+
'signal s_ref: input_'+str(limit)+';\n'+
'signal s_sad_out: std_logic_vector('+str(math.ceil(np.log2(block_size))+bit_width)+' downto 0);')

#################################### Port map ####################################
tb_port_map = ('SAD: '+file_name+' port map (s_ori,s_ref,s_sad_out);') 

#################################### Valores teste ####################################
# tb_value = ('''s_ori(0) <= "0000000000"; -- 0
# 		s_ori(1) <= "0000000000";
# 		s_ori(2) <= "0000000000";
# 		s_ori(3) <= "0000000000";
        
#         s_ref(0) <= "0000000000"; -- 0
# 		s_ref(1) <= "0000000000";
# 		s_ref(2) <= "0000000000";
# 		s_ref(3) <= "0000000000";''')

binario = ['00000000','00001111']
tb_value = []
for i in range(0,limit):
    tb_value.append('s_ori('+str(i)+') <= "'+binario[0]+'";')
for i in range(0,limit):
    tb_value.append('s_ref('+str(i)+') <= "'+binario[1]+'";')
tb_value.append('wait for 10 ns;\n')
for i in range(0,limit):
    tb_value.append('s_ori('+str(i)+') <= "'+binario[1]+'";')
for i in range(0,limit):
    tb_value.append('s_ref('+str(i)+') <= "'+binario[0]+'";')
tb_value.append('wait for 10 ns;\n')
tb_value_concat = '\n'.join(tb_value)
print(tb_value_concat)



s_ori(0) <= "00000000";
s_ori(1) <= "00000000";
s_ori(2) <= "00000000";
s_ori(3) <= "00000000";
s_ori(4) <= "00000000";
s_ori(5) <= "00000000";
s_ori(6) <= "00000000";
s_ori(7) <= "00000000";
s_ori(8) <= "00000000";
s_ori(9) <= "00000000";
s_ori(10) <= "00000000";
s_ori(11) <= "00000000";
s_ori(12) <= "00000000";
s_ori(13) <= "00000000";
s_ori(14) <= "00000000";
s_ori(15) <= "00000000";
s_ref(0) <= "00001111";
s_ref(1) <= "00001111";
s_ref(2) <= "00001111";
s_ref(3) <= "00001111";
s_ref(4) <= "00001111";
s_ref(5) <= "00001111";
s_ref(6) <= "00001111";
s_ref(7) <= "00001111";
s_ref(8) <= "00001111";
s_ref(9) <= "00001111";
s_ref(10) <= "00001111";
s_ref(11) <= "00001111";
s_ref(12) <= "00001111";
s_ref(13) <= "00001111";
s_ref(14) <= "00001111";
s_ref(15) <= "00001111";
wait for 10 ns;

s_ori(0) <= "00001111";
s_ori(1) <= "00001111";
s_ori(2) <= "00001111";
s_ori(3) <= "00001111";
s_ori(4) <= "00001111";
s_ori(5) <= "00001111";
s_ori(6) <= "00001111";
s_ori(7) <= "00001111";
s_ori(8) <=

In [163]:
tb = ('''library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_arith.all;
use work.'''+pack_name+'''.all;

entity '''+ tb_name + ''' is 
end ''' + tb_name +''';

architecture arch_tb of ''' + tb_name + ''' is 
''' + top_component_concat + '''
------------------ Sinais do toplevel -----------------
''' + tb_signal + '''
begin

''' + tb_port_map  + '''

process
begin

''' + tb_value_concat + '''
end process;

end arch_tb;'''

)

print(tb)

library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_arith.all;
use work.type_pack.all;

entity tb_cl_sad_4x4_parallel is 
end tb_cl_sad_4x4_parallel;

architecture arch_tb of tb_cl_sad_4x4_parallel is 
component cl_sad_4x4_parallel is 
 port(
     ori: in input_16;
     ref: in input_16;
     sad_out: out std_logic_vector(12 downto 0)
 );
 end component; 


------------------ Sinais do toplevel -----------------
signal s_ori: input_16;
signal s_ref: input_16;
signal s_sad_out: std_logic_vector(12 downto 0);
begin

SAD: cl_sad_4x4_parallel port map (s_ori,s_ref,s_sad_out);

process
begin

s_ori(0) <= "00000000";
s_ori(1) <= "00000000";
s_ori(2) <= "00000000";
s_ori(3) <= "00000000";
s_ori(4) <= "00000000";
s_ori(5) <= "00000000";
s_ori(6) <= "00000000";
s_ori(7) <= "00000000";
s_ori(8) <= "00000000";
s_ori(9) <= "00000000";
s_ori(10) <= "00000000";
s_ori(11) <= "00000000";
s_ori(12) <= "00000000";
s_ori(13) <= "00000000";
s_ori(14) <= "00000000";
s_ori(15) <= "00000000";
s_ref

## Geração do Package

In [164]:
# # Criando package de tipo
# pack = ('''library ieee;
# use ieee.std_logic_1164.all;
# use ieee.std_logic_unsigned.all;
# use ieee.numeric_std.all;

# package ''' + pack_name + ''' is
    
#     constant bit_width: natural := ''' + str(bit_width) + '''; 
#     type input_line is array (0 to ''' + str(limit-1) + ") of std_logic_vector(" + str(bit_width-1) + ''' downto 0);

# end ''' + pack_name + ';'
# )

# if rewrite_package:
#     pack_file = open((pack_name + ".vhd"), "w")
#     pack_file.write(pack)
#     pack_file.close()